In [43]:
!pip -q install --upgrade pip

In [44]:
!pip -q install --upgrade sagemaker

In [45]:
!pip -q install sagemaker-experiments 

In [46]:
!pip -q install --upgrade sagemaker-huggingface-inference-toolkit

In [47]:
from sagemaker.huggingface import HuggingFaceModel

In [48]:
from sagemaker.analytics import ExperimentAnalytics
import sagemaker
import boto3
from sagemaker.session import Session
from smexperiments.trial_component import TrialComponent

In [49]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [50]:
sess = boto3.Session()
sm = boto3.client('sagemaker')

In [51]:
trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=Session(sess, sm), 
    experiment_name='nlp-classification-1626106647'
)

df_results = trial_component_analytics.dataframe()

In [52]:
df_results.head()

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,epochs,model_name,sagemaker_container_log_level,sagemaker_job_name,sagemaker_program,sagemaker_region,sagemaker_submit_directory,train_batch_size,test:loss - Min,test:loss - Max,test:loss - Avg,test:loss - StdDev,test:loss - Last,test:loss - Count,test:f1 - Min,test:f1 - Max,test:f1 - Avg,test:f1 - StdDev,test:f1 - Last,test:f1 - Count,CrossEntropyLoss_output_0_GLOBAL - Min,CrossEntropyLoss_output_0_GLOBAL - Max,CrossEntropyLoss_output_0_GLOBAL - Avg,CrossEntropyLoss_output_0_GLOBAL - StdDev,CrossEntropyLoss_output_0_GLOBAL - Last,CrossEntropyLoss_output_0_GLOBAL - Count,test:recall - Min,test:recall - Max,test:recall - Avg,test:recall - StdDev,test:recall - Last,test:recall - Count,test:precision - Min,test:precision - Max,test:precision - Avg,test:precision - StdDev,test:precision - Last,test:precision - Count,test:accuracy - Min,test:accuracy - Max,test:accuracy - Avg,test:accuracy - StdDev,test:accuracy - Last,test:accuracy - Count,test - MediaType,test - Value,train - MediaType,train - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments
0,nlp-training-job-distilroberta-base-1626106648...,Training,arn:aws:sagemaker:eu-west-1:905847418383:train...,763104351884.dkr.ecr.eu-west-1.amazonaws.com/h...,1.0,ml.p3.2xlarge,30.0,2.0,"""distilroberta-base""",20.0,"""nlp-training-job-distilroberta-base-1626106648""","""train.py""","""eu-west-1""","""s3://sagemaker-eu-west-1-905847418383/nlp-tra...",32.0,0.172068,0.208827,0.190448,0.025993,0.172068,2,0.926792,0.943036,0.934914,0.011486,0.943036,2,0.699906,0.699906,0.699906,0.0,0.699906,1,0.944633,0.966939,0.955786,0.015773,0.944633,2,0.889846,0.941445,0.915646,0.036486,0.941445,2,0.9233,0.9427,0.9330,0.013718,0.9427,2,None,s3://sagemaker-eu-west-1-905847418383/samples/...,None,s3://sagemaker-eu-west-1-905847418383/samples/...,None,s3://sagemaker-eu-west-1-905847418383/,None,s3://sagemaker-eu-west-1-905847418383/nlp-trai...,[nlp-trial-distilroberta-base-1626106648],[nlp-classification-1626106647]
1,nlp-training-job-distilbert-base-uncased-16261...,Training,arn:aws:sagemaker:eu-west-1:905847418383:train...,763104351884.dkr.ecr.eu-west-1.amazonaws.com/h...,1.0,ml.p3.2xlarge,30.0,2.0,"""distilbert-base-uncased""",20.0,"""nlp-training-job-distilbert-base-uncased-1626...","""train.py""","""eu-west-1""","""s3://sagemaker-eu-west-1-905847418383/nlp-tra...",32.0,0.190021,0.225556,0.207788,0.025127,0.190021,2,0.914150,0.932620,0.923385,0.013060,0.932620,2,0.694372,0.694372,0.694372,0.0,0.694372,1,0.937263,0.958574,0.947919,0.015069,0.937263,2,0.873661,0.928022,0.900842,0.038439,0.928022,2,0.9096,0.9320,0.9208,0.015839,0.9320,2,None,s3://sagemaker-eu-west-1-905847418383/samples/...,None,s3://sagemaker-eu-west-1-905847418383/samples/...,None,s3://sagemaker-eu-west-1-905847418383/,None,s3://sagemaker-eu-west-1-905847418383/nlp-trai...,[nlp-trial-distilbert-base-uncased-1626106647],[nlp-classification-1626106647]


In [53]:
df_results[['model_name', 'test:f1 - Avg', 'test:recall - Avg', 'test:precision - Avg', 'test:accuracy - Avg']]

,model_name,test:f1 - Avg,test:recall - Avg,test:precision - Avg,test:accuracy - Avg
0,"""distilroberta-base""",0.934914,0.955786,0.915646,0.9330
1,"""distilbert-base-uncased""",0.923385,0.947919,0.900842,0.9208


In [54]:
df_results = df_results.sort_values('test:accuracy - Avg', ascending=False)

In [55]:
best_trial_component_name = df_results.iloc[0]["TrialComponentName"]
best_trial_component_name

'nlp-training-job-distilroberta-base-1626106648-aws-training-job'

In [56]:
best_trial_component = TrialComponent.load(best_trial_component_name)
model_data = best_trial_component.output_artifacts["SageMaker.ModelArtifact"].value
model_data

's3://sagemaker-eu-west-1-905847418383/nlp-training-job-distilroberta-base-1626106648/output/model.tar.gz'

In [57]:
role = sagemaker.get_execution_role()
role

'arn:aws:iam::905847418383:role/service-role/AmazonSageMaker-ExecutionRole-20210706T044588'

In [58]:
from sagemaker.huggingface import HuggingFaceModel

In [59]:
huggingface_model = HuggingFaceModel(
    transformers_version="4.6", # transformers version used
    pytorch_version="1.7", # pytorch version used
    py_version="py36", # python version of the DLC
    model_data=model_data,
    role=role,
)

In [60]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

-------------!

In [61]:
sentiment_input= {"inputs": "I love this movie."}
predictor.predict(sentiment_input)

[{'label': 'LABEL_1', 'score': 0.9977279305458069}]

In [62]:
predictor.delete_endpoint()